<a href="https://colab.research.google.com/github/Sugoto/Indic-Language-NLP/blob/main/Hinglish_Script_and_Lang_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import re
import string

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
preModel = "l3cube-pune/hing-bert-lid"
tokenizer = AutoTokenizer.from_pretrained(preModel)
model = AutoModelForTokenClassification.from_pretrained(preModel)
classifier = pipeline("token-classification", model=preModel)

In [12]:
def detect_font(text):
    # Clean the input text
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'[@]', ' ', text)
    text = re.sub(r'[  ]', ' ', text)

    # Detect script
    is_english = any('a' <= char <= 'z' or 'A' <= char <= 'Z' for char in text)
    is_hindi = any(char != " " and not ('a' <= char <= 'z' or 'A' <= char <= 'Z') for char in text)

    if is_english and is_hindi:
        script = "Hinglish mixed"
    elif is_english and not is_hindi:
        script = "English"
    else:
        script = "Hindi"

    # Prepare the text for language detection
    final_text = f'{text.lower()} [SEP]'

    # Chunk the text into smaller segments
    def get_chunks(s, max_length):
        start = end = 0
        while start + max_length < len(s) and end != -1:
            end = s.rfind(" ", start, start + max_length + 1)
            yield s[start:end]
            start = end + 1
        yield s[start:]

    # Detect the language of each chunk
    chunks = get_chunks(final_text, 100)
    chunk_arr = list(chunks)

    word_count = hindi_count = english_count = 0


    for ExampleText in chunkArr:

        res = classifier(ExampleText)

        fw = ''
        si = 1
        for i in range(len(res)):
            sw = res[i]
            word = re.sub(r'[#]', '', sw['word'])
            if sw['index'] > 1:
                if sw['start'] - res[i - 1]['end'] < 2:
                    fw = fw + word
                else:
                    wordCount += 1
                    lang = res[si]['entity']

                    if (ord(fw[0]) > 125):
                        lang = 'HI'
                    if (lang == "HI"):
                        hindiCount = hindiCount + 1
                    else:
                        engCount = engCount + 1
                    fw = word
                    si = i
            else:
                fw = word

    lang = ""
    if hindiCount and engCount:
        lang = "Hinglish mixed"
    elif hindiCount and not engCount:
        lang = "Hindi"
    else:
        lang = "English"

    print(f"Script: {script}")
    print(f"Language: {lang}")
    print(f'Total Number of words in the paragraph: {wordCount}')
    print(f'Number of Hindi words in the paragraph: {hindiCount}')
    print(f'Number of English words in the paragraph: {engCount}')

    data = {'Hindi': hindiCount, 'English': engCount}
    languages = list(data.keys())
    wordcount = list(data.values())


In [13]:
detectFont("mera naam kya hai")

Script: English
Language: Hinglish mixed
Total Number of words in the paragraph: 3
Number of Hindi words in the paragraph: 2
Number of English words in the paragraph: 1
